In [ ]:
import numpy as np
from keras.layers import Input, Dense, Lambda, Concatenate
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import spambase
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess the Spambase dataset
data = spambase.load_data()
X, y = data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert labels to one-hot encoding for classification task
num_classes = 2  # Assuming binary classification for Spambase dataset
y_train_onehot = to_categorical(y_train, num_classes)
y_test_onehot = to_categorical(y_test, num_classes)

# Set up the VAE architecture
original_dim = X_train.shape[1]
latent_dim = 2
intermediate_dim = 256

# Encoder
inputs = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='relu')(inputs)

# Latent space
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Classifier
classifier_h = Dense(128, activation='relu')(z)
classifier_output = Dense(num_classes, activation='softmax')(classifier_h)

# Decoder
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# End-to-end VAE model
vae = Model(inputs, [x_decoded_mean, classifier_output])

# Define the VAE loss
xent_loss = original_dim * objectives.binary_crossentropy(inputs, x_decoded_mean)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
classification_loss = K.categorical_crossentropy(y_train_onehot, classifier_output)

vae_loss = K.mean(xent_loss + kl_loss + classification_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='rmsprop')

# Train the VAE model
vae.fit(X_train, epochs=50, batch_size=64, validation_data=(X_test, [X_test, y_test_onehot]))